This is a quick demo of using Ray to Tune an XGBoost model using a GPU cluster to show utilization of both the driver and worker via ray.tune.

In [0]:
from ray.util.spark import setup_ray_cluster, shutdown_ray_cluster
from ray import train, tune
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn.datasets
import sklearn.metrics
import ray

spark.conf.set('spark.task.resource.gpu.amount', 0)

setup_ray_cluster(
  min_worker_nodes=2,
  max_worker_nodes=2,
  num_cpus_worker_node = 4,
  num_gpus_worker_node = 1,
  num_gpus_head_node = 1,
  num_cpus_head_node= 4
)

# Pass any custom Ray configuration with ray.init
ray.init(ignore_reinit_error=True)
ray.cluster_resources()

In [0]:
def train_breast_cancer(config):
    # Load dataset
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
    # Split into train and test set
    train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.25)

    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)

    # Train the classifier
    results = {}
    xgb.train(
        config,
        train_set,
        num_boost_round=1000000,
        evals=[(test_set, "eval")],
        evals_result=results,
        verbose_eval=False,
    )

    # Return prediction accuracy
    accuracy = 1.0 - results["eval"]["error"][-1]
    train.report({"mean_accuracy": accuracy, "done": True})


config = {
    "objective": "binary:logistic",
    "device": "cuda",
    "eval_metric": ["logloss", "error"],
    "max_depth": tune.randint(1, 9),
    "min_child_weight": tune.choice([1, 2, 3]),
    "subsample": tune.uniform(0.5, 1.0),
    "eta": tune.loguniform(1e-4, 1e-1),
}

tuner = tune.Tuner(
    tune.with_resources(train_breast_cancer, {"gpu": 1}),
    tune_config=tune.TuneConfig(num_samples=10),
    param_space=config,
)

results = tuner.fit()